In [ ]:
import re
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.font_manager
from pprint import pprint

from jieba_postag_converter import get_postag_dict

def get_word_ranking(df,corpus='cejc',situation='apology',sen_type='query', label_col='primitive form', data_col='word count',reverse=False):
    # FOR creating words ranking graph and pos rangling graph
#     cols = {'primitive form':'word count', 'pos':'pos count'}
#     for label_col, data_col in cols.items():
    df[data_col] = df.groupby(['corpus', 'situation', 'method', 'sentence type', 'difference type',label_col])['index'].transform('count')
    df_word = df.drop_duplicates(['corpus', 'situation', 'method', 'sentence type', 'difference type',label_col]).sort_values(by=data_col, ascending=False)
    df_word = df_word.dropna(subset=[label_col])

    # GET data for 1 graph
    difference_type = ['del','add']
    method_type = ['translated','rewrited']
    data, labels = [], []
    
    for diff_type in difference_type:
        _temp_w, _temp_wv = [], [] 
        for method in method_type:
            _temp_w.append(df_word[(df_word['corpus']==corpus)&(df_word['situation']==situation)&(df_word['method']==method)&(df_word['sentence type']==sen_type)&(df_word['difference type']==diff_type)][label_col])
            _temp_wv.append(df_word[(df_word['corpus']==corpus)&(df_word['situation']==situation)&(df_word['method']==method)&(df_word['sentence type']==sen_type)&(df_word['difference type']==diff_type)][data_col])
        # PUT data into dictionary, COUNT freq., CALCULATE (rewrited - translated) and SORT them
        dic = {}
        for i,(wline,vline) in enumerate(zip(_temp_w,_temp_wv)):
            for j,(w,v) in enumerate(zip(wline,vline)):
                if w in dic:
                    dic[w]['each'][i]=v
                    v = v*-1 if i==0 else v
                    dic[w]['diff']=dic.get(w,dic.get('diff',0))['diff']+v
                else:
                    tmp = [0,0]
                    tmp[i] = v
                    v = v*-1 if i==0 else v
                    dic.setdefault(w,{'each':tmp,'diff':v})
        dic = sorted(dic.items(),key=lambda x:x[1]['diff'],reverse=True)
#             pprint(dic)
        # exit()
        words = []
        diff_val=[]
        mt_val=[]
        ht_val=[]
        for key, values in dic:
            words.append(key)
            diff_val.append(values['diff'])
            mt_val.append(values['each'][0])
            ht_val.append(values['each'][1])
#         print(words)
#         print(diff_val)
        def sort_(a,b):
            c = sorted(zip(a, b),reverse=True)
            a=[]
            b=[]
            for aa,bb in c:
                a.append(aa)
                b.append(bb)
            return a, b 
        mt_val,mt_words = sort_(mt_val, words)
        ht_val,ht_words = sort_(ht_val, words)
        
        if label_col == 'pos':
            def convert(pos):
                pos_dic = get_postag_dict()
                for i,p in enumerate(pos):
                    try:
                        pos[i]=pos_dic[p]
                    except:
                        break
                return pos

            mt_words = convert(mt_words)
            ht_words = convert(ht_words)
            words = convert(words)
             
        
        tmp = [mt_words, mt_val, ht_words, ht_val, words, diff_val]
        data.extend(tmp)
#     print(len(data))
#     print(data)
    return data

def get_word_ranking_exe(df,save_fname,obj,cols):
    corpora = ['mpdd','cejc']
    situation_list = ['apology','request','thanksgiving']
    sentence_type_list = ['query','res']

    data = []
    c_name = []
    s_name = []
    t_name = []
    m_name = []
    d_name = []
    r_name = []
    for label_col, data_col in cols.items():
        for corpus in corpora:
            for situation in situation_list:
                for sentence_type in sentence_type_list:
                    tmp = get_word_ranking(df,corpus,situation,sentence_type,label_col,data_col)
                    for i in range(12):
                        c_name.append(f'{corpus}')
                        s_name.append(f'{situation}')
                        t_name.append(f'{sentence_type}')
                    m_name.extend(['MT','MT','HT','HT','Diff','Diff','MT','MT','HT','HT','Diff','Diff'])
                    d_name.extend(['DEL','DEL','DEL','DEL','DEL','DEL','ADD','ADD','ADD','ADD','ADD','ADD'])
                    r_name.extend([obj,'freq',obj,'freq',obj,'freq',obj,'freq',obj,'freq',obj,'freq'])
                    data.extend(tmp)
    #                 print(len(data))
    #                 print(len(c_name))

    tuples = list(zip(c_name,s_name,t_name,m_name,d_name,r_name))
    index = pd.MultiIndex.from_tuples(tuples, names=["corpus", "situation","sentence_type","count_type","unalignment_type","ranking"])
    table = pd.DataFrame(data,index=index)

    dir_name = 'for_thesis/ranking/'
    os.makedirs(dir_name,exist_ok = True) 
    table.to_csv(dir_name+save_fname,encoding='utf_8_sig')


In [ ]:
t_path = "analysis_table.csv"
df = pd.read_csv(t_path)

# ranking
save_fname = 'word.csv'
obj = 'word'
cols = {'primitive form':'word count'}
get_word_ranking_exe(df,save_fname,obj,cols)

save_fname = 'pos.csv'
obj = 'pos'
cols = {'pos':'pos count'}
get_word_ranking_exe(df,save_fname,obj,cols)



# + relation

In [ ]:

def get_word_ranking_in_relation(df,corpus='cejc',situation='apology',sen_type='query', relation='inferior', label_col='primitive form', data_col='word count',reverse=False):
    # FOR creating words ranking graph and pos rangling graph
#     cols = {'primitive form':'word count', 'pos':'pos count'}
#     for label_col, data_col in cols.items():
    df[data_col] = df.groupby(['corpus', 'situation', 'method', 'sentence type', 'difference type','upper relation',label_col])['index'].transform('count')
    df_word = df.drop_duplicates(['corpus', 'situation', 'method', 'sentence type', 'difference type','upper relation',label_col]).sort_values(by=data_col, ascending=False)
    df_word = df_word.dropna(subset=[label_col])

    # GET data for 1 graph
    difference_type = ['del','add']
    method_type = ['translated','rewrited']
    data, labels = [], []
    
    for diff_type in difference_type:
        _temp_w, _temp_wv = [], [] 
        for method in method_type:
            _temp_w.append(df_word[(df_word['corpus']==corpus)&(df_word['situation']==situation)&(df_word['method']==method)&(df_word['sentence type']==sen_type)&(df_word['difference type']==diff_type)&(df_word['upper relation']==relation)][label_col])
            _temp_wv.append(df_word[(df_word['corpus']==corpus)&(df_word['situation']==situation)&(df_word['method']==method)&(df_word['sentence type']==sen_type)&(df_word['difference type']==diff_type)&(df_word['upper relation']==relation)][data_col])
        # PUT data into dictionary, COUNT freq., CALCULATE (rewrited - translated) and SORT them
        dic = {}
        for i,(wline,vline) in enumerate(zip(_temp_w,_temp_wv)):
            for j,(w,v) in enumerate(zip(wline,vline)):
                if w in dic:
                    dic[w]['each'][i]=v
                    v = v*-1 if i==0 else v
                    dic[w]['diff']=dic.get(w,dic.get('diff',0))['diff']+v
                else:
                    tmp = [0,0]
                    tmp[i] = v
                    v = v*-1 if i==0 else v
                    dic.setdefault(w,{'each':tmp,'diff':v})
        dic = sorted(dic.items(),key=lambda x:x[1]['diff'],reverse=True)
#             pprint(dic)
        # exit()
        words = []
        diff_val=[]
        mt_val=[]
        ht_val=[]
        for key, values in dic:
            words.append(key)
            diff_val.append(values['diff'])
            mt_val.append(values['each'][0])
            ht_val.append(values['each'][1])
#         print(words)
#         print(diff_val)
        def sort_(a,b):
            c = sorted(zip(a, b),reverse=True)
            a=[]
            b=[]
            for aa,bb in c:
                a.append(aa)
                b.append(bb)
            return a, b 
        mt_val,mt_words = sort_(mt_val, words)
        ht_val,ht_words = sort_(ht_val, words)
        
        if label_col == 'pos':
            def convert(pos):
                pos_dic = get_postag_dict()
                for i,p in enumerate(pos):
                    try:
                        pos[i]=pos_dic[p]
                    except:
                        break
                return pos

            mt_words = convert(mt_words)
            ht_words = convert(ht_words)
            words = convert(words)
        
        tmp = [mt_words, mt_val, ht_words, ht_val, words, diff_val]
        data.extend(tmp)
#     print(len(data))
#     print(data)
    return data

def get_word_ranking_in_relation_exe(df,save_fname,obj,cols):
    corpora = ['mpdd','cejc']
    situation_list = ['apology','request','thanksgiving']
    sentence_type_list = ['query','res']
    relation_list = ['inferior','peer','inferior']

    data = []
    c_name = []
    s_name = []
    t_name = []
    m_name = []
    d_name = []
    r_name = []
    rel_name = []
    for label_col, data_col in cols.items():
        for corpus in corpora:
            for situation in situation_list:
                for sentence_type in sentence_type_list:
                    for relation in relation_list:
                        tmp = get_word_ranking_in_relation(df,corpus,situation,sentence_type,relation,label_col,data_col)
                        for i in range(12):
                            c_name.append(f'{corpus}')
                            s_name.append(f'{situation}')
                            t_name.append(f'{sentence_type}')
                            rel_name.append(f'{relation}')
                        m_name.extend(['MT','MT','HT','HT','Diff','Diff','MT','MT','HT','HT','Diff','Diff'])
                        d_name.extend(['DEL','DEL','DEL','DEL','DEL','DEL','ADD','ADD','ADD','ADD','ADD','ADD'])
                        r_name.extend([obj,'freq',obj,'freq',obj,'freq',obj,'freq',obj,'freq',obj,'freq'])
                        data.extend(tmp)
#                         print(len(data))
#                         print(len(c_name))

    tuples = list(zip(c_name,s_name,t_name,m_name,d_name,r_name,rel_name))
    index = pd.MultiIndex.from_tuples(tuples, names=["corpus", "situation","sentence_type","count_type","unalignment_type","ranking","relation"])
    table = pd.DataFrame(data,index=index)

    dir_name = 'for_thesis/ranking/'
    os.makedirs(dir_name,exist_ok = True) 
    table.to_csv(dir_name+save_fname,encoding='utf_8_sig')


In [ ]:
save_fname = 'word_in_relation.csv'
obj = 'word'
cols = {'primitive form':'word count'}
get_word_ranking_in_relation_exe(df,save_fname,obj,cols)

save_fname = 'pos_in_relation.csv'
obj = 'pos'
cols = {'pos':'pos count'}
get_word_ranking_in_relation_exe(df,save_fname,obj,cols)

# For Only CONTENT WORDS

In [ ]:
# For Only CONTENT WORDS
t_path = "analysis_table_upper.csv"
df = pd.read_csv(t_path)
# get only content words
content_words = [   '名詞','動詞','形容詞','','','',
                    'a','i','j','l','n','t','v' #形容词 成语 简称略称 习用语 名词 时间词 动词  
                ]
df = df[df['pos'].isin(content_words)]


# ranking
save_fname = 'onlycontents_word.csv'
obj = 'word'
cols = {'primitive form':'word count'}
get_word_ranking_exe(df,save_fname,obj,cols)

save_fname = 'onlycontents_pos.csv'
obj = 'pos'
cols = {'pos':'pos count'}
get_word_ranking_exe(df,save_fname,obj,cols)

save_fname = 'onlycontents_word_in_relation.csv'
obj = 'word'
cols = {'primitive form':'word count'}
get_word_ranking_in_relation_exe(df,save_fname,obj,cols)

save_fname = 'onlycontents_pos_in_relation.csv'
obj = 'pos'
cols = {'pos':'pos count'}
get_word_ranking_in_relation_exe(df,save_fname,obj,cols)